Our best model - Catboost with learning rate of 0.7 and 180 iterations. Was trained on 10 files of the data with similar distribution of the feature user_target_recs (among the number of rows of each feature value). We received an auc of 0.845 on the kaggle leaderboard

#Mount Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#Installations and Imports

In [2]:
# !pip install scikit-surprise
!pip install catboost
# !pip install xgboost

     |████████████████████████████████| 69.2MB 56kB/s 


In [3]:
import os
import pandas as pd
# import xgboost
# from xgboost import XGBClassifier
# import pickle
import catboost
from catboost import CatBoostClassifier

#Global Parameters and Methods

In [4]:
home_path = "/content/drive/MyDrive/RS_Kaggle_Competition"

In [5]:

def get_train_files_paths(path):
  dir_paths = [ os.path.join(path, dir_name) for dir_name in os.listdir(path) if dir_name.startswith("train")]
  file_paths = []

  for dir_path in dir_paths:
    curr_dir_file_paths = [ os.path.join(dir_path, file_name) for file_name in os.listdir(dir_path) ]
    file_paths.extend(curr_dir_file_paths)
  
  return file_paths

train_file_paths = get_train_files_paths(home_path)

#Get Data

In [7]:
def get_df_of_many_files(paths_list, number_of_files):
  for i in range(number_of_files):
    path = paths_list[i]
    curr_df = pd.read_csv(path)

    if i == 0:
      df = curr_df
    else:
      df = pd.concat([df, curr_df])
    
  return df


In [7]:
sample_train_data = get_df_of_many_files(train_file_paths[-21:], 10)
# sample_train_data = pd.read_csv(home_path + "/10_files_train_data")

In [9]:
sample_val_data = get_df_of_many_files(train_file_paths[-10:], 3)
# sample_val_data = pd.read_csv(home_path+"/3_files_val_data")

In [ ]:
# sample_val_data.to_csv(home_path+"/3_files_val_data")

#Preprocess data

In [10]:
train_data = sample_train_data.fillna("Unknown")


In [11]:
val_data = sample_val_data.fillna("Unknown")


In [12]:
train_data

,target_id_hash,syndicator_id_hash,campaign_id_hash,empiric_calibrated_recs,empiric_clicks,target_item_taxonomy,placement_id_hash,user_recs,user_clicks,user_target_recs,publisher_id_hash,source_id_hash,source_item_type,browser_platform,os_family,country_code,region,day_of_week,time_of_day,is_click,day
1,9970,1080,431,129166.300000,103160.0,1,801,43.0,0.0,2.0,2,12801,0,0,6,215,1045,6,21,0.0,19
14,13537,713,11566,2156.825400,8025.0,18,64,585.0,97.0,0.0,0,34217,2,1,2,74,979,5,14,1.0,17
15,31655,511,1745,346.616060,1761.0,7,306,417.0,20.0,0.0,2,45451,1,0,6,215,822,5,15,1.0,17
18,13278,562,8535,5589.715000,28621.0,62,43,874.0,1.0,1.0,0,5633,3,0,6,215,822,5,10,0.0,17
19,38634,67,8091,11604.184000,8411.0,27,330,161.0,5.0,0.0,0,35118,3,0,6,215,840,5,11,0.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155236,11394,585,10546,2673.509000,7028.0,18,1143,111.0,2.0,0.0,0,14337,3,1,3,215,708,6,13,1.0,18
155243,25300,1209,1064,6367.429000,6700.0,55,961,198.0,1.0,0.0,0,49380,3,0,5,215,815,5,8,1.0,17
155252,20111,83,5285,719.816650,3165.0,27,195,334.0,1.0,0.0,0,39662,3,0,6,215,737,0,8,0.0,19
155257,34598,736,12712,15.193707,4.0,1,603,365.0,2.0,0.0,2,12801,0,0,6,215,667,4,20,0.0,17


In [10]:
import gc

del sample_val_data
del sample_train_data
gc.collect()

150

## Scale columns

In [12]:
# scale columns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaling_cols= ["empiric_calibrated_recs", "empiric_clicks", "empiric_calibrated_recs", "user_recs", "user_clicks", "user_target_recs"]

scaler = StandardScaler()

train_data[scaling_cols] = scaler.fit_transform(train_data[scaling_cols])

val_data[scaling_cols] = scaler.transform(val_data[scaling_cols])

In [ ]:
train_data

In [12]:
val_data = val_data.drop(columns=["Unnamed: 0.1"])
val_data

,page_view_start_time,user_id_hash,target_id_hash,syndicator_id_hash,campaign_id_hash,empiric_calibrated_recs,empiric_clicks,target_item_taxonomy,placement_id_hash,user_recs,user_clicks,user_target_recs,publisher_id_hash,source_id_hash,source_item_type,browser_platform,os_family,country_code,region,day_of_week,time_of_day,gmt_offset,is_click
1,1587143260387,cf2e47d172a2049b93954ae8883542c8f8920c21f48696...,c9f6b15564689a1aa31647a7ff9fd7ff9dbc282fc30eed...,daaee2d38f21a75a08d375e55802b3fdfc03cf21608ebd...,46ea1a5cc69bb59fa40f433bb493e8009d78cbe207eb77...,-0.389605,-0.557126,LIFE~HEALTH,77de1750e42c8269e4f36cb4c4be23cef013ceab5cfc7a...,1.940065,0.579458,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,3029c37370f14c987f7feb6ebf0b4bd34ac1e80a4cc03c...,text,DESK,6,US,WA,5,10,-700,0.0
5,1587299786567,b68ff6fffcbd7023cd3b19e6061c92bca031be00f8f7d2...,1bb17d6b92074fa6ce86ed1467fa743557d3d0713c1cb8...,1a080f3235b258f201154f3ec2b37326b6c12cb427a293...,230858acceb89c774ec4e633a9f300348e47ad67678d92...,-0.375228,-0.553739,TECH,0802dbe49f63a78283c0fae1100dbe6fb87e2eff72c6d0...,-0.940284,-0.510636,-0.244194,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,c83159630182a5b7bc3c3fc985156c03b2a43129d63376...,homepage,DESK,6,PR,UNKNOWN,0,8,-400,0.0
14,1587221676604,230feaf700acffef21360153bf668c417e23c42c96c663...,474bfb9df176c068cbe422517fb5f93258fe5ad0f7787b...,467e05fdef8657c4a81c79501cbeb4744c1f2400e36c34...,2bf350b3644f97debc4e04565b8d08f01229d7cbea9d23...,-0.370608,-0.503520,LIFE,082bbc538cc7b5b016cc88289c4b8aee2fa5e3963dfa77...,-0.178498,0.782266,-0.244194,c29a980222a2a97b74ffbd067f2a4ce760e3e8d8f02494...,4c7987d659b97d2007c45bb3c9fd903b135a2ef18130f7...,homepage,DESK,5,US,AZ,6,7,-700,1.0
17,1587089254787,fe53cdb2b0879abf559b4082f46bd56c37e2ab2c61d21a...,37e0294e44419197f54ac395adf56c0cab2826702cd358...,5ab40b3661279f6dd915b3d5a1dab0c2152449601bdb23...,d6e6d2d06565e38beb9efaa53ad85b12c864924fcdd927...,-0.388489,-0.550609,SPORTS,8844d7d2d7a66ee65b40604e119ab9b1ce96fe6ef335db...,-0.833236,-0.459934,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,1066ebdc08c9d8d39ac23be406b66733f7d4fea8176c63...,text,DESK,5,US,LA,4,21,-500,1.0
19,1587150366962,9214888421a799f66b6b84f7507e60edfde2e95c08837d...,e39e1ab2b0f6039551429fb0a63889a2eb6e1fa520061a...,00969476791e522c12df362a27e2c03e99223b3679f233...,ea6262ed2f507396ece6c21678f7e33bf3e0b916d8e749...,2.559099,1.590994,BUSINESS,932fbd7955800fd5a36f071a7e74f18343c263cf4bd950...,-0.803362,-0.459934,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,bd01bba4cac969422c933a17854178afb81ab0d00e8fab...,text,PHON,2,US,TX,5,14,-500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4183234,1587141615499,524a3c9431969137b90cf201b854f663edb9a65756af82...,a121414eede8630b38ec7368707f9ef8771616118bf3c1...,139d1b7d4e3d66843ff0fffcda0e073a5a7b0ec050338b...,55e54997c2099f1eed1084a1f14698c20d33ef71da31f1...,5.560015,0.950711,TECH,1fd2398ac133f366d026494f1ffa35c8bdd16a39648bef...,-0.285546,-0.510636,23.487108,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,389b69708cf58731e78e2e023a8c608d886fe5aebc0e23...,category,DESK,6,US,AR,5,11,-500,0.0
4183239,1587162891251,bab7651bdf76e1a1143baf47e4e1cdb36f8c5e3ba12186...,f781e7ba2d4bc85105ea2a0fda2607797b9a92eafa6d64...,c77fba0111e87134c253255e726d94cba6731e7f63b0a0...,4eb7845122f855487e2a811236fb39f41fe15174ccb975...,-0.321318,-0.544817,BUSINESS,7c5a99a7413dc26705a689d6466b3e7f857d83036c7ebb...,2.746663,-0.434583,23.487108,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,95d8fe9299e2bdf5ca092e8bce601fbc7d3cf89bb11035...,text,DESK,6,US,PA,5,18,-400,0.0
4183247,1587089062833,d289b898160d5bd4d1aaa76b8f023f34de2d52bddd19e7...,a121414eede8630b38ec7368707f9ef8771616118bf3c1...,139d1b7d4e3d66843ff0fffcda0e073a5a7b0ec050338b...,55e54997c2099f1eed1084a1f14698c20d33ef71da31f1...,5.368597,0.904113,TECH,6055091bd2f1c62e48ee3cb89c8e7f0829edc6a05e9f94...,-0.302973,-0.510636,23.487108,f8a7ba9b7c9b05464ee98daac

#Explore Data

In [ ]:
sample_train_data

In [ ]:
test_data

In [ ]:
from collections import Counter
user_recs_dist = test_data["user_recs"].value_counts(normalize=True)

top_user_recs_count = user_recs_dist.nlargest(200)

print(top_user_recs_count)

percent = sum(top_user_recs_count.values)
percent

In [ ]:
print(sample_train_data["user_recs"].value_counts(normalize=False))
print(test_data["user_recs"].value_counts())

0.0       542238
300.0      50285
3.0        46951
1.0        45815
2.0        39176
           ...  
3262.0         1
3795.0         1
3793.0         1
3788.0         1
3224.0         1
Name: user_recs, Length: 3947, dtype: int64
0.0       112851
3.0        10185
1.0        10088
2.0         8581
4.0         6510
           ...  
2190.0         1
2549.0         1
2720.0         1
2289.0         1
2588.0         1
Name: user_recs, Length: 2534, dtype: int64


In [ ]:
positions = top_user_recs_count
def sample(obj, replace=False, total=1500000):
    return obj.sample(n=int(positions[obj.name] * total), replace=replace)

sample_train_data_filtered = sample_train_data[sample_train_data["user_recs"].isin(positions.keys())]
result = sample_train_data_filtered.groupby("user_recs").apply(sample).reset_index(drop=True)
result["user_recs"].value_counts(normalize=True)

0.0      0.339258
3.0      0.030618
1.0      0.030327
2.0      0.025796
4.0      0.019570
           ...   
310.0    0.001024
200.0    0.001022
348.0    0.001018
199.0    0.001018
183.0    0.001016
Name: user_recs, Length: 200, dtype: float64

In [ ]:
top_user_recs_train_data = result

In [ ]:
top_user_recs_train_data

In [ ]:
not_top_user_recs_train_data = sample_train_data[~sample_train_data["user_recs"].isin(top_user_recs_train_data["user_recs"].unique())]
not_top_user_recs_train_data["user_recs"].value_counts()

NameError: ignored

In [ ]:
train_data = pd.concat([top_user_recs_train_data, not_top_user_recs_train_data])

In [ ]:
train_data["user_recs"].value_counts(normalize=False)

0.0       380819
3.0        34369
1.0        34042
2.0        28956
4.0        21968
           ...  
3276.0         1
3772.0         1
3766.0         1
3765.0         1
3176.0         1
Name: user_recs, Length: 3947, dtype: int64

In [ ]:
train_data = train_data.drop(columns = ["user_id_hash"])

In [ ]:
train_data = train_data.fillna("Unknown")

In [ ]:
train_data

,page_view_start_time,target_id_hash,syndicator_id_hash,campaign_id_hash,empiric_calibrated_recs,empiric_clicks,target_item_taxonomy,placement_id_hash,user_recs,user_clicks,user_target_recs,publisher_id_hash,source_id_hash,source_item_type,browser_platform,os_family,country_code,region,day_of_week,time_of_day,gmt_offset,is_click
0,1586808490113,bc494fe2722c59b5a0181848f9ec210f6d42a4b2ab91dd...,9c802dae4e25f48705de3b06421e48a2c3838732ed0c40...,4cb3516cefd443bd8ea224385630f23960bfd0f9d615cd...,7582.14060,3811.0,UNKNOWN,7e0924571d90e13aba2b85370672b96830cfd830b0fdba...,0.0,0.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,955388548eb686a42cabaa2869cdcc6a43cb1bd2ef57fe...,text,DESK,6,US,VA,1,16,-400,0.0
1,1587043849187,b9096f867957d2ef02a12c1b9dcd34c7172d2d94b7ae04...,82d07b44ce8d5ec2db9f1e407585b5bc671e7496ec6819...,2ed997ff869378f813dcd868e0dc03f861dd0b0c08b1d4...,613.46020,288.0,TECH,291d3ff037c1ea0beb8e1ca27e1383031d3bd4e4671f4b...,0.0,0.0,0.0,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,c83159630182a5b7bc3c3fc985156c03b2a43129d63376...,homepage,DESK,6,US,TX,4,8,-500,0.0
2,1586788268609,f0d7e2009602acc14d2d760757dba91eab94bb1d934aa4...,daaee2d38f21a75a08d375e55802b3fdfc03cf21608ebd...,80cfde501e22775c6a5d23fa08ae3b414593149e876d9a...,111.71033,684.0,LIFE~HEALTH,f1b3343d5e6491da91d7c65d1ad3140bb1983eedb1ed2d...,0.0,0.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,a6587d9c09add99edc45d84628a1ad93e7df47904a650a...,text,DESK,6,US,IN,1,10,-400,1.0
3,1586214596459,6719de5d8f315f40533c465f51c077467292ce00796587...,bc7cd65002a6888221920604bbc306a3e40def36e1f093...,8dc2dd75130bce36fb69deb6295e62280746d056d9e19a...,1131.66400,587.0,BUSINESS,3c1c8ea94e66caefc804eba418aa893f0d8fc361ad86e5...,0.0,0.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,973ccc00a59de8f1bee9a42fb663487e4dd07eea010b7c...,text,DESK,6,US,NY,1,19,-400,0.0
4,1586309747543,d6c0318e03e48af275c64666a6501108d5bd6c2777cce1...,74a92474ac4530441ffadf8ca1f982a949417b95ab80e6...,c774a4e8cd46baefea9f73928f820fd234642530a58c76...,102720.47000,107819.0,BUSINESS,b1c0e2baa89e19c667a4c77835b2b1ccbbe87960f6a64b...,0.0,0.0,0.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,111f21461365664321b2e0a252105ca2099e00a2107003...,text,DESK,6,US,CA,2,18,-700,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437337,1586725728226,cd8af33d9f39e9a5b327fc80aca7084941eefb0dccf665...,f7887a181154b8238e77e1f8ff155461e81683526e534f...,296ad477193175c09697f825949052b4d4910d9ee2be86...,1427.40150,1187.0,BUSINESS,77de1750e42c8269e4f36cb4c4be23cef013ceab5cfc7a...,776.0,43.0,2.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,2b1fb5f13f7bfa5b2b6e633154415f1c959906a743e098...,text,DESK,6,US,KY,0,16,-500,0.0
437338,1586967723919,8db4a0d386fe637106a5832eedab9117740f8acfee7be1...,0a5ae6b5f1cafa40b71081d4ab114b1c5ae607b0a0d6b3...,8587cd0a6b690ab76e487e5df5f440fbe6ce05f20b5b3e...,1464.09400,709.0,LIFE~HEALTH,736ce0a301eb0c3da05800df13949e411684c97236aa22...,523.0,7.0,2.0,c29a980222a2a97b74ffbd067f2a4ce760e3e8d8f02494...,4c7987d659b97d2007c45bb3c9fd903b135a2ef18130f7...,homepage,DESK,6,US,CA,3,9,-700,0.0
437339,1586699384200,b404117ca60c567340a42fbdf7efabebeb943fe75ce8cd...,daaee2d38f21a75a08d375e55802b3fdfc03cf21608ebd...,f31cf7d913662ded88821b9aea265e3f1eab56bed81619...,4190.96700,34500.0,LIFE~HEALTH,89c5fcd2164ba85b038e7136967b91e824054bb8194da2...,368.0,52.0,0.0,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,c83159630182a5b7bc3c3fc985156c03b2a43129d63376...,homepage,DESK,6,US,ID,0,7,-600,1.0
437340,1586874595514,8c5f4fe19554e6b7fbf6cb49846a6eddfa21d0c13808a8...,e0f1fd8a33b136275228e3558b5cf78f25dd880b6eab84...,3d58bf024dfc5c5f027bf927b47319bb22a93e80542505...,5339.62650,3793.0,UNKNOWN,3241b9ff5f1128f35690a6403d79f8a66434455fb5060f...,883.0,20.0,1.0,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,317eda02f21cb5673a247574568e3c0738d56770fad501...,text,DESK,6,US,OH,2,10,-400,0.0


#Train the model

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics


X_train = train_data.drop(columns=["is_click"], inplace=False)
y_train = train_data["is_click"]

X_val = val_data.drop(columns=["is_click"], inplace=False)
y_val = val_data["is_click"]

In [14]:
from catboost import CatBoostClassifier
# cat_features_inds = [1,2,3,4,7,8,12,13,14,15,17,18]
encode_cols = [ "user_id_hash", "target_id_hash", "syndicator_id_hash", "campaign_id_hash", "target_item_taxonomy", "placement_id_hash", "publisher_id_hash", "source_id_hash", "source_item_type", "browser_platform", "country_code", "region", "gmt_offset"]

# model = CatBoostClassifier(iterations = 50, learning_rate=0.5, task_type='CPU', loss_function='Logloss', cat_features = encode_cols)
model = CatBoostClassifier(iterations = 180, learning_rate=0.7, task_type='CPU', loss_function='Logloss', cat_features = encode_cols,
                           eval_metric='AUC')#, depth=6, l2_leaf_reg= 10)

"""
All of our tries with catboost (only the best of them were uploaded to kaggle):

results:
all features, all rows of train fillna = Unknown
logloss 100 iterations learning rate 0.5 10 files: 0.857136889762303 | bestTest = 0.4671640673 0.857136889762303
logloss 100 iterations learning rate 0.4 10 files: bestTest = 0.4676805926 0.856750110976787

logloss 100 iterations learning rate 0.55 10 files: bestTest = 0.4669830858 0.8572464626142212

logloss 120 iterations learning rate 0.6 10 files: bestTest = 0.4662084678 0.8577564702279399

logloss 150 iterations learning rate 0.7 10 files: bestTest = 0.4655981391 0.8581645278496352

logloss 180 iterations learning rate 0.7 10 files: bestTest = 0.4653168207 0.8583423138228865 !!!!!!!!!!

logloss 180 iterations learning rate 0.7 10 files day extracted from date (not as categorical): 0.8583034988

logloss 180 iterations learning rate 0.7 10 files day extracted from date (as categorical): 0.8583014151

logloss 180 iterations learning rate 0.75 10 files day extracted from date (as categorical): 0.8582889749

logloss 180 iterations learning rate 0.65 10 files day extracted from date (as categorical): 0.8582334254

logloss 180 iterations learning rate 0.65 10 files day extracted from date (as categorical) StandardScaler: 0.8582101013

logloss 180 iterations learning rate 0.7 10 files day extracted from date (as categorical) MinMaxScaler dropna: ~0.8582

logloss 180 iterations learning rate 0.7 distributed data train and val, day extracted as categorical MinMaxScaler: 0.8561707 

logloss 180 iterations learning rate 0.7 distributed data train and val, day extracted as not categorical no scale: 0.8561707195

logloss 180 iterations learning rate 0.7 distributed data train and val, no scale no date: 0.8559952294

logloss 180 iterations learning rate 0.7 distributed data train and val, day extracted as not categorical no scale with date: 0.8560461554

logloss 180 iterations learning rate 0.7, 9 times distributed data train and val, no user no date: 0.8545560094

logloss 180 iterations learning rate 0.7, 9 times distributed data train and val, with user and numeric day: 0.8561601034

logloss 180 iterations learning rate 0.7, 9 times distributed data train and val, with user with numeric date: 0.8568834122

logloss 180 iterations learning rate 0.7, 10 different files, scaled, all features: 0.8584829166 !!!!!!!

logloss 180 iterations learning rate 0.7, new data, scaled, all features: 0.8915972905 test: 0.84108


logloss 180 iterations learning rate 0.9 10 files: bestTest = 0.4656462845

logloss 100 iterations learning rate 0.5 8 files: 0.8568031111965864
logloss 300 iterations learning rate 0.5: 
crossentropy 50 iterations learning rate 0.5: 0.8556282878645277
"""

'\nresults:\nall features, all rows of train fillna = Unknown\nlogloss 100 iterations learning rate 0.5 10 files: 0.857136889762303 | bestTest = 0.4671640673 0.857136889762303\nlogloss 100 iterations learning rate 0.4 10 files: bestTest = 0.4676805926 0.856750110976787\n\nlogloss 100 iterations learning rate 0.55 10 files: bestTest = 0.4669830858 0.8572464626142212\n\nlogloss 120 iterations learning rate 0.6 10 files: bestTest = 0.4662084678 0.8577564702279399\n\nlogloss 150 iterations learning rate 0.7 10 files: bestTest = 0.4655981391 0.8581645278496352\n\nlogloss 180 iterations learning rate 0.7 10 files: bestTest = 0.4653168207 0.8583423138228865 !!!!!!!!!!\n\nlogloss 180 iterations learning rate 0.7 10 files day extracted from date (not as categorical): 0.8583034988\n\nlogloss 180 iterations learning rate 0.7 10 files day extracted from date (as categorical): 0.8583014151\n\nlogloss 180 iterations learning rate 0.75 10 files day extracted from date (as categorical): 0.8582889749\n

In [15]:
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=10)

0:	test: 0.8149026	best: 0.8149026 (0)	total: 6.36s	remaining: 18m 57s
10:	test: 0.8461028	best: 0.8461028 (10)	total: 53.6s	remaining: 13m 44s
20:	test: 0.8490288	best: 0.8490288 (20)	total: 1m 38s	remaining: 12m 26s
30:	test: 0.8505695	best: 0.8505695 (30)	total: 2m 23s	remaining: 11m 29s
40:	test: 0.8514950	best: 0.8514950 (40)	total: 3m 8s	remaining: 10m 38s
50:	test: 0.8522340	best: 0.8522340 (50)	total: 3m 53s	remaining: 9m 50s
60:	test: 0.8526374	best: 0.8526374 (60)	total: 4m 37s	remaining: 9m
70:	test: 0.8531463	best: 0.8531463 (70)	total: 5m 22s	remaining: 8m 14s
80:	test: 0.8534035	best: 0.8534035 (80)	total: 6m 6s	remaining: 7m 27s
90:	test: 0.8536159	best: 0.8536567 (89)	total: 6m 51s	remaining: 6m 42s
100:	test: 0.8537674	best: 0.8537674 (100)	total: 7m 35s	remaining: 5m 56s
110:	test: 0.8539636	best: 0.8539636 (110)	total: 8m 19s	remaining: 5m 10s
120:	test: 0.8541628	best: 0.8541628 (120)	total: 9m 3s	remaining: 4m 25s
130:	test: 0.8542642	best: 0.8542642 (130)	total: 9

# Submission File

In [16]:
test_data = pd.read_csv("/content/drive/MyDrive/RS_Kaggle_Competition/test/test_file.csv")


In [17]:
test_data = test_data.iloc[:,:-1]

In [18]:
test_data[scaling_cols] = scaler.transform(test_data[scaling_cols])

In [19]:

X_test = test_data.fillna("Unknown")
X_test

,page_view_start_time,user_id_hash,target_id_hash,syndicator_id_hash,campaign_id_hash,empiric_calibrated_recs,empiric_clicks,target_item_taxonomy,placement_id_hash,user_recs,user_clicks,user_target_recs,publisher_id_hash,source_id_hash,source_item_type,browser_platform,os_family,country_code,region,day_of_week,time_of_day,gmt_offset
0,1587359560809,26bd0dea68c5f65c286ead5f6a69218486cc965535ae39...,79f2e0095aca1d3afac3a4918f2fc77aa3dd386537447a...,377402b6b9107d4c2d3d7cd40218ab1578ffd15554a2c7...,26443482ee16a2b3653fab3cbf84eb2c3931520a985db5...,-0.370496,-0.389020,SPORTS~FIGHTING,911f042701f83ed9e2c3ee0951dd2f9e1dc0302ca5a03a...,-0.733656,-0.510636,-0.244194,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,c83159630182a5b7bc3c3fc985156c03b2a43129d63376...,homepage,DESK,6,US,AZ,0,22,-700
1,1587341629681,f183602e0fe86592a106e62230d05ecc02212be60def80...,1077d1cd2590b9318ce5bf1b63f323706d9b3734234640...,fe55e950d29575f0dd1a02958cedaeb271ca167482c1bc...,1b40924dcbe3fe73d7ba308be724d722c6279ace116e78...,0.528702,1.073877,LIFE~HEALTH-AND-FITNESS,736ce0a301eb0c3da05800df13949e411684c97236aa22...,-0.793404,-0.510636,-0.244194,c29a980222a2a97b74ffbd067f2a4ce760e3e8d8f02494...,4c7987d659b97d2007c45bb3c9fd903b135a2ef18130f7...,homepage,DESK,6,US,IN,0,20,-400
2,1587406996730,a35e998230d1c5a2dd0ee62004dd0dfe47b0fd29b1e20c...,9a4de107996cf896aed161e9579b1384464b363aae7e2c...,fe55e950d29575f0dd1a02958cedaeb271ca167482c1bc...,b94436fabe57b43407e63bbf23cdeccb237603b6104425...,0.229005,1.046572,LIFE~HEALTH,3df5c61ea574ff3b3df24ce8d6479af393d1df05048150...,-0.927837,-0.510636,-0.244194,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,389b69708cf58731e78e2e023a8c608d886fe5aebc0e23...,category,DESK,6,US,OH,1,14,-400
3,1587426992796,c0901569facec5fbe96f16a15c7674d57da91916a82d72...,770a38ba68a2b9a6647d8db845fc68ff9bd3e0d09b7c8b...,26fb5310eed046f76c8ab6ea27e96b275771bd9f49c107...,e154a93347c6f190739b5d454cbbb7a623bd6f32242332...,-0.386593,-0.557990,UNKNOWN,2a252fe72b34a7a10c784178e6def75b3e7226194daf95...,-0.902942,-0.383881,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,b0c391512c4c4829dd44b4c6429a4b4e474d0a61f55f8a...,text,PHON,3,US,NY,1,19,-400
4,1587408832707,18c2e5a4263f4464b096a056ad14d04aa901c9bc8ac0a7...,79f2e0095aca1d3afac3a4918f2fc77aa3dd386537447a...,377402b6b9107d4c2d3d7cd40218ab1578ffd15554a2c7...,26443482ee16a2b3653fab3cbf84eb2c3931520a985db5...,-0.368447,-0.374328,SPORTS~FIGHTING,a5bc8458778dcec7656c4bfa2045322e0042ca7ea4b84d...,-0.290525,-0.510636,-0.244194,f8a7ba9b7c9b05464ee98daac522b3f3d2376453c70e57...,389b69708cf58731e78e2e023a8c608d886fe5aebc0e23...,category,DESK,6,US,VA,1,14,-400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444501,1587411981514,19f77f9adc386080d3c4beb92416f64d7f97d6ae0ecb12...,04be22dcff0409d33dbd8ccf6aa4b9f55d3499ff8f5e3e...,c9553679452b5d0d95f5eb5644584da5095d5306a3d990...,5a4683f74c746e7c674c2dc65584de2a936623fe3eef1c...,-0.348855,-0.264687,LIFE~FOOD,fa632cbb6db6d4eb17c9ad466e1fa291bcc6f0c3e3bb93...,-0.594244,-0.358530,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,f6650be39f0a1b2c9397a8a4a595fa649b984b1d6a1665...,text,PHON,2,US,AR,1,14,-500
444502,1587391464693,e50ea8cecaed72a92992a1a3442396965d5292c3328007...,ebba98b926c360c393666b68ce923c64b8432352bb299d...,47063015027cbfcd0346a3ba47d3a8e06d59149c3db1fe...,32b9f7b92c4bb45e5f15ef677b6b7ff0ee90fa228518de...,-0.343122,-0.456384,LIFE~HEALTH,b1c0e2baa89e19c667a4c77835b2b1ccbbe87960f6a64b...,-0.868089,-0.510636,-0.244194,848b127dbd0f3a647581f4a95325f5204f4577ad7956f6...,db749b9ac4e230f4b9cc0b464509d8c91a91f47396cd37...,text,DESK,6,US,FL,1,10,-400
444503,1587418949929,919f0bfc8fc58362803c223028c465d60fd29bc318d175...,e589fe763cb444e7d4d51bd8add68e34e2dae45f9b44d4...,d9c04132bce53c7de71c1b2f3f415460986f053a088bf3...,a1df50275ae35e7c2ed6167e4e7066b19cb76129eac8c4...,-0.390450,-0.562896,LIFE~HEALTH,8236a1925848dcb75cfe2979cd41516fb78c1cd303e957...,-0.823278,-0.510636,-0.244194,848b127dbd0f

In [20]:
pred_proba = model.predict_proba(X_test)

submission_dir_path = "/content/drive/MyDrive/RS_Kaggle_Competition/submissions"
pred = pred_proba[:,1]

pred_df = pd.DataFrame(pred)
pred_df.reset_index(inplace=True)
pred_df.columns = ['Id', 'Predicted']
pred_df.to_csv(submission_dir_path + '/catboost_submission_datafrom1704_data_lr_0.7_with_scale_with_num_startdate_with_user_iters_159.csv', index=False)